## LangChain chains

In [1]:
!pip install -q langchain langchain-openai python-dotenv

### Prosty chain: prompt → model → wynik

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_messages([
    ("system", "jesteś ekspertem programowania w Pythonie i geniuszem w dziedzinie AI."),
    ("user", "napisz kod {topic}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()
result = chain.invoke({'topic': 'AGI'})

print(result)

Tworzenie prawdziwej sztucznej inteligencji ogólnej (AGI) to niezwykle złożone zadanie, które wykracza poza możliwości obecnych technologii i narzędzi. AGI odnosi się do systemów AI, które mogą rozumieć, uczyć się i stosować wiedzę w różnych dziedzinach, podobnie jak człowiek. Obecnie nie istnieje żaden kod, który mógłby być uznany za AGI.

Jednak mogę pokazać ci prosty przykład, jak można zbudować system AI, który uczy się na podstawie danych. Poniżej znajduje się przykład prostego modelu uczenia maszynowego w Pythonie, który wykorzystuje bibliotekę `scikit-learn` do klasyfikacji danych.

```python
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Wczytanie zbioru danych (np. Iris)
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X

### Sequential chain: dwa modele w sekwencji

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Chain: streszczenie (wejście: {tekst} → wyjście: str)
summary_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Streszcz poniższy tekst w 1–2 zdaniach po polsku."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

# 2) Adapter: zamień str → {"tekst": str}, żeby podać do następnego promptu
to_dict = RunnableLambda(lambda s: {"tekst": s})

# 3) Chain: tłumaczenie streszczenia na francuski (wejście: {tekst} → wyjście: str)
translate_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Przetłumacz tekst na język francuski."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

# 4) Prawdziwy sequential chain: summary → (wrap) → translate
sequential_chain = summary_chain | to_dict | translate_chain

# Test — jedno wywołanie całego łańcucha
input_text = "LangChain umożliwia tworzenie aplikacji AI poprzez łączenie modeli, promptów i narzędzi w spójne pipeline’y."
final_translation = sequential_chain.invoke({"tekst": input_text})

print("Tłumaczenie francuskie:\n", final_translation)



Tłumaczenie francuskie:
 LangChain permet de créer des applications d'IA en intégrant des modèles, des invites et des outils dans des processus organisés.


### Branching chain: jedna odpowiedź, dwa przetworzenia

In [9]:
from langchain_core.runnables import RunnableParallel

# Prompt do streszczenia
prompt_summary = ChatPromptTemplate.from_template("Streszcz: {tekst}")

# Prompt do sentymentu
prompt_sentiment = ChatPromptTemplate.from_template("Określ ton wypowiedzi: {tekst}")

# Dwa równoległe przetworzenia
branch_chain = RunnableParallel(
    summary=(prompt_summary | llm | StrOutputParser()),
    sentiment=(prompt_sentiment | llm | StrOutputParser()),
)

text = "Jestem bardzo zadowolony z tego kursu, nauczyłem się dużo o LangChain!"
result = branch_chain.invoke({"tekst": text})

print(result)


{'summary': 'Uczestnik kursu wyraża swoje zadowolenie, podkreślając, że zdobył wiele wiedzy na temat LangChain.', 'sentiment': 'Ton wypowiedzi jest pozytywny i entuzjastyczny. Osoba wyraża zadowolenie z kursu oraz podkreśla, że zdobyła cenną wiedzę na temat LangChain.'}


### Prosty RAG chain

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

docs = [
    "LangChain to framework do pracy z dużymi modelami językowymi.",
    "Chains w LangChain to przepływy danych pomiędzy promptami, modelami i parserami.",
    "Retriever pozwala wyszukiwać informacje w bazie wektorowej."
]

splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
splitted = splitter.split_text(" ".join(docs))

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(splitted, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Odpowiedz tylko na podstawie KONTEKSTU:\n{context}"),
    ("user", "{question}")
])

def format_docs(docs):
    return "\n".join(d.page_content for d in docs)

runnable = RunnablePassthrough.assign(context=lambda x: x["question"])

rag_chain = (
    {
        "context": runnable | (lambda x: x["question"]) | retriever | format_docs,
        "question": lambda x: x["question"]
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke({"question" : "Do czego służy biblioteka LangChain?"})

print(response)


Biblioteka LangChain służy do pracy z dużymi modelami językowymi, umożliwiając tworzenie przepływów danych pomiędzy promptami, modelami i parserami. Dodatkowo, LangChain pozwala na wyszukiwanie informacji w bazie wektorowej.
